# IMMC 2020
### Importing Modules and Data

In [14]:
import pandas as pd
import copy
import random
import math, statistics
import matplotlib.pyplot as plt
from pprint import pprint

median_income = 9733/365
loss_aversion_coefficient = 2

pdt_csv_data = pd.read_csv("StoreData_IMMC_CSV.csv")

In [15]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

## Determining the Popularity of Product
### Impact of Discount on Popularity

In [16]:
# Traditional Econs Approach
def q1_over_q0(p0, p1, percentage_usage):
    x = (p0 - p1)*(percentage_usage/median_income) 
    return (math.exp(x))

# Behavioural Econs Approach
def prospect_utility(x):
    if x > 0:
        return(math.log(x+1))
    else:
        return(-loss_aversion_coefficient * math.log(-x + 1))

def increase_utility(p0, p1):
    return(prospect_utility(p0 - p1))

# Helper Variables
max_increase_utility = increase_utility(3329.99, 2199.99)
min_increase_utility = 0
max_q1_over_q0 = q1_over_q0(3329.99, 2199.99, 1)
min_q1_over_qo = 1

# Combined Effect [Between 0 and 1]
def popularity_due_to_discount(p0, p1, percentage_usage):
    traditional_econs_adjusted = (q1_over_q0(p0, p1, percentage_usage)-min_q1_over_qo)/max_q1_over_q0
    behavioural_econs_adjusted = (increase_utility(p0, p1)-min_increase_utility)/max_increase_utility

    total_adjusted = statistics.mean([traditional_econs_adjusted, behavioural_econs_adjusted])
    return(total_adjusted)

### Effect of Loss Adversion on Popularity

In [17]:
# [Between 0 and 1]
def popularity_due_to_loss_aversion(qty):
    return(math.exp(-qty/loss_aversion_coefficient))

### Effects of Saliency Bias on Popularity (TO DO)

In [18]:
def popularity_due_to_saliency_bias(size, qty):
    return sigmoid(size*qty)

### Effects of Ratings on Popularity

In [19]:
# [Between 0 and 1]
def popularity_due_to_rating(pdt_rating, brand_rating):
    raw_brand = (0.8*pdt_rating + 0.2*brand_rating)
    return(raw_brand/5)

## Creating Product Class and List

In [20]:
pdt_list = []
class product:
    # Popularity Coefficients
    pop_loss_adversion_coefficient = 0.33
    pop_saliency_coefficient = 0.33
    pop_rating_coefficient = 0.33
    
    
    # Raw Data
    def __init__(self, index, name, department, product_category, product_type, brand, initial_price, discounted_price, qty, customer_rating, brand_rating=5, percentage_usage=0.5, size=20):
        self.name = name
        self.index = index
        self.department = department
        self.product_category = product_category
        self.product_type = product_type
        self.brand = brand
        self.initial_price = initial_price
        self.discounted_price = discounted_price
        self.qty = qty
        self.customer_rating = customer_rating
        self.brand_rating = brand_rating
        self.percentage_usage = percentage_usage
        self.size = size
        
        self.popularity = 0
    
    # Processed Data
    
    def set_popularity(self):
        discount_factor = popularity_due_to_discount(self.initial_price, self.discounted_price, self.percentage_usage)
        loss_adversion_factor = popularity_due_to_loss_aversion(self.qty)
        saliency_factor = popularity_due_to_saliency_bias(self.size, self.qty)
        rating_factor = popularity_due_to_rating(self.customer_rating, self.brand_rating)
        
        initial_popularity = self.pop_loss_adversion_coefficient*loss_adversion_factor + self.pop_saliency_coefficient*saliency_factor + self.pop_rating_coefficient*rating_factor
        self.popularity = sigmoid(initial_popularity + discount_factor)
        
        
        

In [21]:
# Populating the Product List
for index, row in pdt_csv_data.iterrows():
    cur_pdt = product(index, row["name"], row["department"], row["product_category"], row["product_type"], row["brand"], row["initial_price"], row["discounted_price"], row["qty"], row["customer_rating"])
    cur_pdt.set_popularity()
    
    pdt_list.append(cur_pdt)


In [22]:
pdt_list[4].index

4

## Creating Shelf Class and Layout Object

In [23]:
class shelf:
    # Class Variables
    max_capacity = 100
    
    def __init__(self):
        self.pdts = []
        self.pdt_set = set()
        self.cur_capacity = 0
    
    def add_pdt(self, pdt):
        # If shelf can accomodate the product
        if self.cur_capacity + pdt.size <= self.max_capacity:
            self.cur_capacity += pdt.size
            self.pdts.append(pdt)
            self.pdt_set.add(pdt.index)
            return 0
        
        # If shelf is full
        return -1

In [24]:
class layout:
    
    def __init__(self, grid, counter, shelf_list=[], pdt_list=[]):
        self.counter = counter
        self.entrance = [7,7]
        self.exit = [7,6]
        
        # Grid is a 2d matrix where shelves are 1 indexed
        self.grid = copy.deepcopy(grid)
        self.shelf_list = copy.deepcopy(shelf_list)
        
        # A* Grid is a grid where shelves are labelled as 1
        self.a_star_grid = copy.deepcopy(grid)
        for i in range(len(self.a_star_grid)):
            for j in range(len(self.a_star_grid)):
                if self.a_star_grid[i][j] > 1:
                    self.a_star_grid[i][j] = 1
                    
        # Shopper Density Grid is a grid to record the density of shoppers; shelves have a density of 99
        self.shopper_density_grid = copy.deepcopy(self.a_star_grid)
        for i in range(len(self.shopper_density_grid)):
            for j in range(len(self.shopper_density_grid)):
                if self.shopper_density_grid[i][j] == 1:
                    self.shopper_density_grid[i][j] = -1
                    
        # Price Density Grid is a grid to record to value of products the customers are carrying at particular locations
        self.price_density_grid = copy.deepcopy(grid)
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                if self.price_density_grid[i][j] > 0:
                    self.price_density_grid[i][j] = 0
                    
                    
        # pdt_list contains the products that exists somewhere within the layout
        self.pdt_list = copy.deepcopy(pdt_list)
        
        
        
    # A_Star Performs a simulation of a person walking within the layout from init to goal
    # A_Star Returns a list of nodes visited on the path
    # Coordinates are written as [y,x] with [0,0] being the upper left hand corner
    def a_star(self, init, goal): 
        grid = copy.deepcopy(self.a_star_grid)
        
        # init = [0, 0]
        # goal = [len(grid) - 1, len(grid[0]) - 1]  # all coordinates are given in format [y,x]
        
        cost = 1

        # the cost map which pushes the path closer to the goal
        heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
                if grid[i][j] == 1:
                    heuristic[i][j] = 99  # added extra penalty in the heuristic map


        # the actions we can take
        delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right


        # function to search the path
        def search(grid, init, goal, cost, heuristic):

            closed = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the reference grid
            closed[init[0]][init[1]] = 1
            action = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the action grid

            x = init[0]
            y = init[1]
            g = 0
            f = g + heuristic[init[0]][init[0]]
            cell = [[f, g, x, y]]

            found = False  # flag that is set when search is complete
            resign = False  # flag set if we can't find expand

            while not found and not resign:
                if len(cell) == 0:
                    return "FAIL"
                else:
                    cell.sort()  # to choose the least costliest action so as to move closer to the goal
                    cell.reverse()
                    next = cell.pop()
                    x = next[2]
                    y = next[3]
                    g = next[1]

                    if x == goal[0] and y == goal[1]:
                        found = True
                    else:
                        for i in range(len(delta)):  # to try out different valid actions
                            x2 = x + delta[i][0]
                            y2 = y + delta[i][1]
                            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                                if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                                    g2 = g + cost
                                    f2 = g2 + heuristic[x2][y2]
                                    cell.append([f2, g2, x2, y2])
                                    closed[x2][y2] = 1
                                    action[x2][y2] = i
            invpath = []
            x = goal[0]
            y = goal[1]
            invpath.append([x, y])  # we get the reverse path from here
            while x != init[0] or y != init[1]:
                x2 = x - delta[action[x][y]][0]
                y2 = y - delta[action[x][y]][1]
                x = x2
                y = y2
                invpath.append([x, y])

            path = []
            for i in range(len(invpath)):
                path.append(invpath[len(invpath) - 1 - i])
                
#             print("ACTION MAP")
#             for i in range(len(action)):
#                 print(action[i])

            return path
        return search(grid, init, goal, cost, heuristic)
    
    # Simulates Choice of Object to Buy and Deletes that Object from Object List
    def choose_pdt(self):
        #print("pdt list len", len(self.pdt_list))
        if len(self.pdt_list) == 0:
            print("RAN OUT OF ITEMS IN SHOP")
            return -1
        self.pdt_list.sort(key=lambda x: x.popularity, reverse=True)
        index = min(random.randint(0, 5), len(self.pdt_list) -1)
        chosen_pdt = copy.deepcopy(self.pdt_list[index])
        if self.pdt_list[index].qty == 1:
            self.pdt_list.pop()
        else:
            self.pdt_list[index].qty -= 1
        
        return chosen_pdt
    
    # Find Product in Sheleves
    def find_shelf(self, chosen_pdt_index):
        for shelf_index, shelf in enumerate(self.shelf_list):
                if chosen_pdt_index in shelf.pdt_set:
                    #print("Found Object")
                    chosen_shelf_index = shelf_index

                    # Remove product from shelf
                    
                    for i in range(len(shelf.pdts)):
                        if shelf.pdts[i].index == chosen_pdt_index:
                            if shelf.pdts[i].qty <= 1:
                                del shelf.pdts[i]
                                shelf.pdt_set.discard(chosen_pdt_index)
                            else:
                                shelf.pdts[i].qty-= 1
                            break
                        
                    break
        return chosen_shelf_index
        
    def walk(self, init, goal, cur_size, cur_price):
        # Conduct A*
        delta = [(1,0), (-1,0), (0,1), (0,-1)]
        for d in delta:
            y = goal[0]+d[0]
            x = goal[1]+d[1]
            if x >= 0 and x < len(self.grid) and y >= 0 and y < len(self.grid[0]):
                if goal == self.exit:
                    [y,x] = [goal[0], goal[1]]
                path = self.a_star(init, [y,x])
                # print("init", init)
                # print("goal", [y,x])
                if path == "FAIL":
                    z=0
                    #print("FAILURE TO REACH SHELF by going to ", [y,x])
                else:
                    for cell in path:
                        #print("walked", cur_size)
                        self.shopper_density_grid[cell[0]][cell[1]] += cur_size
                        self.price_density_grid[cell[0]][cell[1]] += cur_price
                    cur_pos = [y,x]
                    print("Visited ", y, x)
                    return cur_pos
        
    # Simulates a person entering shop
    def new_shopper(self):
        print("--Testing New Shopper--")
        max_number_of_products = 3
        max_size = 300
        
        cur_number_of_pdt = 0
        cur_size = random.randint(40, 60)
        cur_price = 0
        
        print("Size of Shopper:", cur_size)
        
        cur_pos = self.entrance
        while cur_size < max_size and cur_number_of_pdt < 3 and len(self.pdt_list) > 0:
            
            # Choose what Product to Buy
            chosen_pdt = self.choose_pdt()
            if cur_size + chosen_pdt.size > max_size:
                print("next pdt too heavy")
                break
            
            if chosen_pdt == -1:
                print("Ending Simulation")
                return
            print("Chosen Product:", chosen_pdt.index)
        
            # Find Shelf Index
            chosen_shelf_index = self.find_shelf(chosen_pdt.index)
            print("Shelf Containing Product:", chosen_shelf_index)
        
            # Find Location of Shelf 
            for y in range(len(self.grid)):
                for x in range(len(self.grid[0])):
                    if self.grid[y][x] == chosen_shelf_index:
                        shelf_location = (y,x)
    
            # Walk from cur_pos to another shelf while tracking the movement of the shopper
            cur_pos = self.walk(cur_pos, shelf_location, cur_size, cur_price)
            cur_number_of_pdt +=1
            cur_price += chosen_pdt.initial_price
            cur_size += chosen_pdt.size
        
        # Walk to Counter
        cur_pos = self.walk(cur_pos, self.counter, cur_size, cur_price)
        
        # Walk to Exit [cur_price == 0 because they aready paid]
        cur_pos = self.walk(cur_pos, self.exit, cur_size, 0)
        print("--Finishing Shopper--")
                
        return copy.deepcopy(self.shopper_density_grid)
        
    def get_price_grid(self):
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                self.price_density_grid[i][j] = int(self.price_density_grid[i][j])
        return copy.deepcopy(self.price_density_grid)

In [25]:
## Testing the Layout Object
example_layout_grid = [
    [0,0,0,0,0,0,0,0],
    [0,0,1,0,0,2,0,0],
    [0,0,3,0,0,4,0,0],
    [0,0,11,0,0,6,0,0],
    [0,0,12,0,0,7,0,0],
    [0,0,13,0,0,8,0,0],
    [0,0,14,0,0,9,0,0],
    [0,0,15,0,0,10,0,0]
]

pdt_1 = copy.deepcopy(pdt_list[0])
pdt_2 = copy.deepcopy(pdt_list[1])

total_value = pdt_1.qty*pdt_1.discounted_price + pdt_2.qty*pdt_2.discounted_price
print("total_value", total_value)

shelf_0 = shelf()
shelf_1 = shelf()
shelf_2 = shelf()

shelf_1.add_pdt(pdt_1)
shelf_2.add_pdt(pdt_2)

counter = (7,0)
test_layout = layout(example_layout_grid, counter, [shelf_0, shelf_1, shelf_2], [pdt_1, pdt_2])

shopper_density_grid = test_layout.new_shopper()
shopper_density_grid = test_layout.new_shopper()

price_density_grid = test_layout.get_price_grid()

pprint(shopper_density_grid)
print("---- Price Grid ----")
pprint(price_density_grid)

total_value 4999.9
--Testing New Shopper--
Size of Shopper: 50
Chosen Product: 0
Shelf Containing Product: 1
Visited  0 2
Chosen Product: 0
Shelf Containing Product: 1
Visited  0 2
Chosen Product: 0
Shelf Containing Product: 1
Visited  0 2
Visited  6 0
Visited  7 6
--Finishing Shopper--
--Testing New Shopper--
Size of Shopper: 54
Chosen Product: 0
Shelf Containing Product: 1
Visited  0 2
Chosen Product: 0
Shelf Containing Product: 1
Visited  0 2
Chosen Product: 1
Shelf Containing Product: 2
Visited  0 5
Visited  6 0
Visited  7 6
--Finishing Shopper--
[[224, 448, 880, 536, 536, 536, 328, 104],
 [224, 224, -1, 0, 0, -1, 224, 104],
 [224, 224, -1, 0, 0, -1, 224, 104],
 [224, 224, -1, 0, 0, -1, 224, 104],
 [224, 224, -1, 0, 0, -1, 224, 104],
 [224, 224, -1, 0, 0, -1, 224, 104],
 [448, 224, -1, 0, 0, -1, 224, 104],
 [0, 0, -1, 0, 0, -1, 224, 104]]
---- Price Grid ----
[[4079, 4079, 8159, 3399, 3399, 3399, 0, 0],
 [4079, 0, 0, 0, 0, 0, 0, 0],
 [4079, 0, 0, 0, 0, 0, 0, 0],
 [4079, 0, 0, 0, 0,

## Calculating Product Damage
### Collision Damage / Self-Drops

In [26]:
def loss(shopper_density, price_density):
    p_collision = shopper_density / (300*300)
    return p_collision*price_density

def total_loss(shopper_density_grid, price_density_grid):
    net_loss = 0
    for y in range(len(shopper_density_grid)):
        for x in range(len(price_density_grid)):
            net_loss += loss(shopper_density_grid[y][x], price_density_grid[y][x])

    return net_loss

print(total_loss(shopper_density_grid, price_density_grid))

242.02746666666667
